In [1]:
import math
import ast
import turtle
import random
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import statistics
from statistics import mean
population_size = 10000
max_nodes = 50

In [2]:
def draw(angles):
    num_sides = len(angles)
    ext_angles = [180 - angle for angle in angles]
    turtle.setup(500, 500)
    turtle.speed(0)
    turtle.pensize(3)
    turtle.hideturtle()
    for angle in ext_angles:
        turtle.forward(100)
        turtle.right(angle)
    turtle.done()

In [3]:
def population_generation(population_size):
    chromosomes = []
    for i in range(population_size):
        chromosome = []
        nodes = random.randint(1, max_nodes)
        chromosome.append(nodes)
        for j in range(nodes):
            angle = random.randint(0, 180)
            chromosome.append(angle)
        chromosomes.append(chromosome)   
    return chromosomes

In [4]:
def fitness_function(chromosomes):
    interior_sum = 720
    interior_angle = 120
    equality = []
    angle_difference = []
    fitness_scores = []
    
    for index, chromosome in enumerate(chromosomes):
        temp = []
        for i in range(len(chromosome)):
            if (i > 0):
                temp.append(1 - (abs(chromosome[i] - interior_angle) / interior_angle))
        equality.append(sum(temp) / len(temp))
        
    for chromosome in chromosomes:
        sum_of_interior_angles = sum(chromosome[1:])
        maximum = 180 * max_nodes
        angle_difference.append(1 - abs(sum_of_interior_angles - interior_sum) / maximum)
        
    for k in range(len(equality)):
        fitness_scores.append((equality[k] + angle_difference[k]) / 2)
    fitness_scores = [round(score, 5) for score in fitness_scores]
    
    return fitness_scores

In [5]:
def selection(chromosomes, fitness_scores):
    chromosomes_with_scores = list(zip(chromosomes, fitness_scores))
    sorted_chromosomes = sorted(chromosomes_with_scores, key = lambda x: x[1], reverse = True)
    
    num_to_select = int(len(sorted_chromosomes) * 0.2) 
    selected_chromosomes = [chromosome for chromosome, score in sorted_chromosomes[:num_to_select]]
    
    random.shuffle(selected_chromosomes)
    return selected_chromosomes

In [6]:
def crossover(parents):
    random.shuffle(parents)
    children = []
    if((random.randint(1, 10) / 10) > 0.5): #50% chance
        for i in range(0, len(parents) - 1, 2):
            parent1 = parents[i]
            parent2 = parents[i + 1]
            if (len(parent1) > len(parent2)):
                parent1_g1 = parent1[:len(parent2)]
                parent1_g2 = parent1[len(parent2):]

                child1 = parent1_g1
                child1[0] = len(parent2) - 1

                child2 = parent2 + parent1_g2 
                child2[0] = len(parent1) - 1
            elif (len(parent1) < len(parent2)):
                parent2_g1 = parent2[:len(parent1)]
                parent2_g2 = parent2[len(parent1):]

                child1 = parent2_g1
                child1[0] = len(parent1) - 1

                child2 = parent1 + parent2_g2
                child2[0] = len(parent2) - 1
            else:
                if (len(parent1) % 2 == 0):
                    child1 = parent1[:int(len(parent1)/2)] + parent2[int(len(parent2)/2):]
                    child2 = parent2[:int(len(parent2)/2)] + parent1[int(len(parent1)/2):]
                elif (len(parent1) % 2 == 1):
                    child1 = parent1[:int(len(parent1)/2)] + parent2[1 + int(len(parent2)/2):]
                    child2 = parent2[:int(len(parent2)/2)] + parent1[1 + int(len(parent1)/2):]
            children.append(child1)     
            children.append(child2)
    else:  
        for i in range(0, len(parents) - 1, 2): #50% chance
            parent1 = parents[i]
            parent2 = parents[i + 1]
            if (len(parent1) > len(parent2)):
                for j in range(len(parent2)):
                    if (j % 2 == 1):
                        temp = parent2[j]
                        parent2[j] = parent1[j]
                        parent1[j] = temp
            elif (len(parent1) < len(parent2)):
                for j in range(len(parent1)):
                    if (j % 2 == 1):
                        temp = parent1[j]
                        parent1[j] = parent2[j]
                        parent2[j] = temp
            else:
                for j in range(len(parent1)):
                    if (j % 2 == 1):
                        temp = parent1[j]
                        parent1[j] = parent2[j]
                        parent2[j] = temp
            children.append(parent1)
            children.append(parent2)
    random.shuffle(children)
    return children

In [7]:
def mutation(chromosomes):
    for chromosome in chromosomes:
        if ((random.randint(1, 10) / 10) < 0.2): #20% chance
            index = random.randint(1, len(chromosome) - 1)
            chromosome[index] = random.randint(0, 180)
    return chromosomes

In [8]:
def bestchromosome(chromosomes, fitness_scores):
    max_value = max(fitness_scores)
    max_index = fitness_scores.index(max_value)
    return {str(chromosomes[max_index]):max_value}

In [9]:
def replacement(chromosomes, fitness_scores, children):
    #ONLY REPLACE IF THE AVERAGE FITNESS OF CHILDREN IS MORE THAN THAT OF THE CHROMOSOMES
    if (sum(fitness_function(children))/len(children) > sum(fitness_function(chromosomes))/len(chromosomes)):
        chromosomes_with_scores = list(zip(chromosomes, fitness_scores))
        sorted_chromosomes = sorted(chromosomes_with_scores, key = lambda x: x[1], reverse = True)

        num_to_select = int(len(sorted_chromosomes) * 0.8) 
        selected_chromosomes = [chromosome for chromosome, score in sorted_chromosomes[:num_to_select]]

        new_generation = selected_chromosomes + children
        random.shuffle(new_generation)
        return new_generation
    else:
        return chromosomes

In [10]:
chromosomes = population_generation(population_size)
fitness_scores = fitness_function(chromosomes)
best_chromosomes = []
generations = 1000
for i in range(generations):
    parents = selection(chromosomes, fitness_scores)
    children = crossover(parents)
    children = mutation(children)
    chromosomes = replacement(chromosomes, fitness_scores, children)
    random.shuffle(chromosomes)
    fitness_scores = fitness_function(chromosomes)
    best_chromosomes.append(bestchromosome(chromosomes, fitness_scores))
    print("Generation", i + 1, "best chromosome", bestchromosome(chromosomes, fitness_scores))
    if (list(best_chromosomes[i].values())[0] == 1.0):
        break
max_pair = max(best_chromosomes, key = lambda x: list(x.values())[0])
draw(list(ast.literal_eval(list(max_pair.keys())[0]))[1:])

Generation 1 best chromosome {'[2, 120, 125]': 0.96319}
Generation 2 best chromosome {'[1, 120]': 0.96667}
Generation 3 best chromosome {'[1, 121]': 0.96256}
Generation 4 best chromosome {'[2, 118, 120]': 0.96906}
Generation 5 best chromosome {'[2, 120, 120]': 0.97333}
Generation 6 best chromosome {'[4, 119, 118, 123]': 0.97167}
Generation 7 best chromosome {'[5, 120, 143, 120, 119, 122]': 0.973}
Generation 8 best chromosome {'[6, 121, 120, 123, 122, 130, 128]': 0.982}
Generation 9 best chromosome {'[6, 114, 132, 120, 119, 122, 121]': 0.98428}
Generation 10 best chromosome {'[5, 121, 116, 120, 119, 122]': 0.98656}
Generation 11 best chromosome {'[5, 121, 120, 119, 121, 116]': 0.98733}
Generation 12 best chromosome {'[6, 120, 120, 120, 122, 122, 121]': 0.99625}
Generation 13 best chromosome {'[6, 120, 120, 120, 122, 122, 121]': 0.99625}
Generation 14 best chromosome {'[6, 120, 118, 120, 119, 122, 120]': 0.99647}
Generation 15 best chromosome {'[6, 120, 120, 120, 122, 121, 121]': 0.997}
